# Text Summarization with T5

This notebook covers the end-to-end process of fine-tuning a T5-small model on the CNN/DailyMail dataset for text summarization.

## 1. Environment Setup

In [ ]:
!pip install transformers datasets rouge_score py7zr accelerate evaluate nltk pandas matplotlib

In [ ]:
import torch
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

nltk.download('punkt')

## 2. Dataset Collection

In [ ]:
# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Optimization: Use a smaller subset for faster training
print("Slicing dataset for faster training...")
dataset["train"] = dataset["train"].select(range(2000))
dataset["validation"] = dataset["validation"].select(range(500))

print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")
print(f"Test: {len(dataset['test'])}")

# Show a sample
sample = dataset['train'][0]
print(f"\nArticle:\n{sample['article'][:500]}...")
print(f"\nHighlights:\n{sample['highlights']}")

## 3. Data Exploration

In [ ]:
# Analyze token lengths to determine max_input_length and max_target_length
# Taking a subset for speed if needed, but here's the logic

train_df = pd.DataFrame(dataset['train'].select(range(1000))) # select first 1000 for quick analysis

train_df['article_len'] = train_df['article'].apply(lambda x: len(x.split()))
train_df['highlights_len'] = train_df['highlights'].apply(lambda x: len(x.split()))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.hist(train_df['article_len'], bins=50)
ax1.set_title('Article Word Count')
ax2.hist(train_df['highlights_len'], bins=50)
ax2.set_title('Summary Word Count')
plt.show()

## 4. Data Preprocessing

In [ ]:
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 1024
max_target_length = 128
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

## 5. Model Setup

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
batch_size = 16
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## 6. Model Training

In [ ]:
args = Seq2SeqTrainingArguments(
    f"{model_checkpoint}-finetuned-cnn",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True, # Enable mixed precision for faster training on GPU
)

# Metric setup
metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

## 7. Model Evaluation

In [ ]:
trainer.evaluate()

## 8. Save Model

In [ ]:
trainer.save_model("./my_summarization_model")